In [21]:
import notebook_utils as nu
import scipy.io
from pathlib import Path
import numpy as np
import os
import cv2
from tqdm import tqdm
import copy

In [ ]:
shape_template = {
    "label": "",
    "points": [],
    "group_id": None,
    "shape_type": "polygon",
    "flags": {}
}

label_file_template = {
    "version": "4.5.6",
    "flags": {},
    "shapes": [],
    "imagePath": "",
    "imageData": None,
    "imageHeight": 240,
    "imageWidth": 320
}

### Mat file conversion

In [3]:
polygon_label_folder= "TextSnake.pytorch/dataset/total_text/Groundtruth/Polygon/Train"
rect_label_folder= "TextSnake.pytorch/dataset/total_text/Groundtruth/Polygon/Train"

image_folder = "TextSnake.pytorch/dataset/total_text/Images/Train"

mat_files = nu.list_files(polygon_label_folder, "*.mat")
image_files = nu.list_files(image_folder, "*.jpg")

In [10]:
def mat_to_json_conversion(mat_dict, image_name, img_size):
    label_dict = copy.deepcopy(label_file_template)
    label_dict["imagePath"] = image_name
    label_dict["imageHeight"] = img_size[0]
    label_dict["imageWidth"] = img_size[1]
    
    poly_dict = mat_dict["polygt"]
    for i in range(len(poly_dict)):
        x = poly_dict[i][1][0]
        y = poly_dict[i][3][0]
        label = poly_dict[i][4][0]
        
        points = np.stack((x,y), 1).astype(np.float32).tolist()
    
        instance = copy.deepcopy(shape_template)
        instance["label"] = label
        instance["points"] = points

        label_dict["shapes"].append(instance)
        
    return label_dict

In [ ]:
os.makedirs(output_dir)

In [19]:
output_dir="TextSnake.pytorch/dataset/total_text/Label/Train"

output_image_dir = "../../Images/Train"

for file in tqdm(mat_files):
#     file = mat_files[1]
    mat = scipy.io.loadmat(file)
    name = file.stem.split("_")[-1]
    image_name = name + ".jpg"
    
    image_file =  Path(image_folder) / image_name
    
    if not image_file.is_file():
        print("File not found")
        continue
    
    image_size = cv2.imread(str(image_file)).shape[:2]
    
    try:
        json = mat_to_json_conversion(mat, str(Path(output_image_dir) / image_name), image_size)
    except:
        print("Error conversion")
    
    nu.write_json(Path(output_dir) / (name + ".json"), json)

In [20]:
nu.zip("TextSnake.pytorch/dataset/total_text/Label/", "totaltext_label")

### Txt conversion

In [87]:
def txt_to_json_conversion(txt_list, image_name, img_size):
    label_dict = copy.deepcopy(label_file_template)
    label_dict["imagePath"] = image_name
    label_dict["imageHeight"] = img_size[0]
    label_dict["imageWidth"] = img_size[1]
    
    for i in range(len(txt_list)):
        l = txt_list[i].strip().split(',')
#         print(l)
        label = ",".join(l[8:])
        points = np.array(l[:8]).astype(np.float32).reshape(4, 2).tolist()
        
        instance = copy.deepcopy(shape_template)
        instance["label"] = label
        instance["points"] = points

        label_dict["shapes"].append(instance)
        
    return label_dict

In [88]:
txt_label_folder= "datasets/Coop_export/full_image"
image_folder = "datasets/ReceiptCOOPData_v0.1_image"

txt_files = nu.list_files(txt_label_folder, "*.txt")

output_image_dir = "../ReceiptCOOPData_v0.1_image"
output_dir="datasets/ReceiptCOOPData_v0.1_labels_2"

for file in tqdm(txt_files):
#     file = mat_files[1]
    with open(file, 'r') as f:
        lines = f.readlines()
    
    name = file.stem
    image_name = name + ".jpg"
    
    image_file =  Path(image_folder) / image_name
    
    if not image_file.is_file():
        print("File not found")
        continue
    
    image_size = cv2.imread(str(image_file)).shape[:2]
    
#     try:
    json = txt_to_json_conversion(lines, str(Path(output_image_dir) / image_name), image_size)
#     except:
#         print("Error conversion")
    
    nu.write_json(Path(output_dir) / (name + ".json"), json)

100%|██████████| 1105/1105 [02:11<00:00,  8.41it/s]


In [89]:
nu.zip(output_dir, "receiptCoop_label2")

In [80]:
lines[0].strip().lower().split(',')

['3183', '162', '3423', '216', '3365', '470', '3125', '416', 'rn']

In [77]:
",".join(lines[0].split(",")[8:]).replace("\n", "")

'rn'

In [ ]:
with open(str(file_path.with_suffix('.txt')), 'r') as f:
    lines = f.readlines()